# Amazon Forecast チュートリアル -電気使用量の予測-

## Table Of Contents
1. [はじめに](#はじめに)
1. [セットアップ](#セットアップ)
1. [データの準備](#データの準備)
1. [データセットグループとデータセットの作成](#データセットグループとデータセットの作成)
1. [Predictorの作成](#Predictorの作成)
1. [Forecastの作成](#Forecastの作成)
1. [予測値と実測値の可視化](#予測値と実測値の可視化)
1. [RollingForecastの実行](#RollingForecastの実行)
1. [リソースの削除](#リソースの削除)

## はじめに

本ノートブックの実行には約1.5hほどかかります。

本ノートブックはSageMakerからAmazon Forecastの操作を行う[amazon-forecast-samples](https://github.com/aws-samples/amazon-forecast-samples)のTutorialに加え、Rolling Forecast(既存のPredictorを使用して、Predictorを再学習することなく、さらに先のデータポイントの予測を行う)の実行を含んでいます。

本ノートブックの実行には、AmazonSageMakerのIAMロールにs3にアクセスできる権限付与（ノートブックインスタンス作成時に行います）と「IAMFullAccess」、「AmazonForecastFullAccess」のポリシーをアタッチする必要があります。    
また、本ノートブック上で作成したs3バケットを削除したい場合は「AmazonS3FullAccess」もアタッチする必要があります(s3バケットの削除はs3のコンソールからも行えますので「AmazonS3FullAccess」は必須ではありません)。

boto3を使用したForecastの操作については[ForecastService](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/forecast.html)にドキュメントがあります。

## セットアップ

必要なライブラリのインポートと分析データをアップロードするs3バケットを指定します。

In [ ]:
import sys
import os
import json
import time
import logging

import dateutil.parser
import pandas as pd
import boto3
from botocore.exceptions import ClientError

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("./common") )
import util

分析に使用するデータを格納するs3のバケットとregionを表示されるフォーム内に入力してください。    
すでに存在するバケット名でも新規に作成するバケット名でも問題ありません。    
バケットを新規に作成する際はs3のバケットの命名規則に従う必要があります。

In [ ]:
text_widget_bucket = util.create_text_widget( "bucket_name", "input your S3 bucket name" )
text_widget_region = util.create_text_widget( "region", "input region name.", default_value="us-west-2" )

フォームへ入力が完了したら以下のセルを実行してください。

In [ ]:
bucket_name = text_widget_bucket.value
assert bucket_name, "bucket_name not set."

region = text_widget_region.value
assert region, "region not set."

In [ ]:
session = boto3.Session(region_name=region) 
forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

## データの準備

Forecastの[Getting Started](https://docs.aws.amazon.com/forecast/latest/dg/getting-started.html)からダウンロードできる電気使用量の[データ](https://docs.aws.amazon.com/forecast/latest/dg/samples/electricityusagedata.zip)を使用します。
このデータには370世帯の2014/01/01から2014/12/31（2015/01/01の0時まで）までの1年分の電力使用量が毎時単位で格納されています。

In [ ]:
!wget https://docs.aws.amazon.com/forecast/latest/dg/samples/electricityusagedata.zip
!unzip -o electricityusagedata.zip

データを読み込み、3つのファイルを作成します。ファイルの作成はダウンロードしたelectricityusagedata.csvを使用し、各対象の期間を抽出します。
- electricityusagedata_train.csvは、学習用データとしてpredictorを作成するために利用します。この Predictor で予測を行うと、2014/12/30 01:00:00から Forecast Horizon で指定した期間 (本ノートブックでは2014/12/31 00:00:00までの24時間分) の予測を得ることができます。
- 本ノートブックでは、electricityusagedata_train.csvに24時間分のデータを追加した、electricityusagedata_add.csvを利用することでRolling Forecastを実行し、2014/12/31 00:00:00より先の予測を行います。
- electricityusagedata_test.csvは実測値としてRolling Forecastの予測値との比較に使用します。

In [ ]:
df = pd.read_csv("./electricityusagedata.csv", dtype = object, names=['timestamp','value','item'])
df.head()

In [ ]:
train_data = df[(df['timestamp'] >= '2014-01-01') & (df['timestamp'] <= '2014-12-30 00:00:00')]

add_data = df[(df['timestamp'] >= '2014-01-01') & (df['timestamp'] <= '2014-12-31 00:00:00')]

test_data = df[(df['timestamp'] >= '2014-01-01') & (df['timestamp'] <= '2015-01-01 00:00:00')]

In [ ]:
train_data.tail()

In [ ]:
add_data.tail()

In [ ]:
test_data.tail()

In [ ]:
train_data.to_csv("electricityusagedata_train.csv", header=False, index=False)
add_data.to_csv("electricityusagedata_add.csv", header=False, index=False)
test_data.to_csv("electricityusagedata_test.csv", header=False, index=False)

s3のバケットを作成し、データをアップロードします。すでに作成しているバケットを使用する際も以下のコードを実行して問題ありません。    
バケットの新規作成が行われる場合はcreate_bucket()のreturnがTrueとなります。    
バケットがすでに作成済みであればcreate_bucket()のreturnがFalseになり、作成は行われません。

In [ ]:
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
create_bucket(bucket_name=bucket_name, region=region)

バケットにデータをアップロードします。

In [ ]:
# 学習用データ
key="electricityusagedata_train.csv"
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file("electricityusagedata_train.csv")

# Rolling Forecast用のデータ
add_key="electricityusagedata_add.csv"
boto3.Session().resource('s3').Bucket(bucket_name).Object(add_key).upload_file("electricityusagedata_add.csv")

## データセットグループとデータセットの作成

In [ ]:
DATASET_FREQUENCY = "H" 
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"

In [ ]:
# 作成するデータセットグループやデータセットに使用する接頭辞です。任意の名前を入力してください。
project = 'forecast_tutorial'

# データセットグループ名
datasetGroupName= project +'_dsg'
# データセット名
datasetName= project+'_ds'

# 学習用データのS3パス
s3DataPath = "s3://"+bucket_name+"/"+key

データセットグループを作成(定義)します。    

In [ ]:
create_dataset_group_response = forecast.create_dataset_group(
    DatasetGroupName=datasetGroupName,
    Domain="CUSTOM"
)

datasetGroupArn = create_dataset_group_response['DatasetGroupArn']

In [ ]:
forecast.describe_dataset_group(DatasetGroupArn=datasetGroupArn)

データセットのスキーマを作成(定義)します。

In [ ]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }
   ]
}

データセットを作成(定義)します。

In [ ]:
response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema=schema
)

In [ ]:
datasetArn = response['DatasetArn']
forecast.describe_dataset(DatasetArn=datasetArn)

データセットグループにデータセットを追加します。    
本分析では「分析対象のデータセットのみ」を使用しますが、データセットグループには分析対象のデータセットのほか、「メタデータ」、「関連時系列データ」を格納でき、分析対象のデータセットと併せることでモデルの予測精度の改善が見込めます。

In [ ]:
forecast.update_dataset_group(DatasetGroupArn=datasetGroupArn, DatasetArns=[datasetArn])

### Forecast用のIAMロールを作成する
以下のコードの実行にはSageMakerのロールにIAMFullAccessの権限を追加する必要があります。

多くのAWSサービスと同様に、ForecastはS3リソースと安全にやり取りするためにIAMロールを引き受ける必要があります。
本ノートブックでは、get_or_create_iam_role（）ユーティリティ関数を使用してIAMロールを作成します。 実装については、
["common/util/fcst_utils.py"](./common/util/fcst_utils.py)を参照してください。

In [ ]:
# Create the role to provide to Amazon Forecast.
role_name = "ForecastNotebookRole-Tutorial"
role_arn = util.get_or_create_iam_role(role_name=role_name)

### Data Import Jobの作成

一連のデータセット関連の定義が完了したので、s3からAmazon Forecastにデータをインポートします。

In [ ]:
datasetImportJobName = 'my_dsimportjob'
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [ ]:
ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(ds_import_job_arn)

以下のセルに実行には約5-10分かかります。

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

In [ ]:
forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)

## Predictorの作成

「forecastHorizon(予測期間)」は、将来予測される時点の数です。 週次データの場合、12と入力すると12週を意味します。 今回のデータは時間ごとのデータです。次の日の予測を試みるので、24に設定します。

In [ ]:
# predictorの名前
predictorName= project+'_ets_algo'

In [ ]:
# 予測期間
forecastHorizon = 24

In [ ]:
# アルゴリズム（他のアルゴリズムに変更することも可能です）
algorithmArn = 'arn:aws:forecast:::algorithm/ETS'

In [ ]:
create_predictor_response=forecast.create_predictor(PredictorName=predictorName, 
                                                  AlgorithmArn=algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                         "BackTestWindowOffset": 24}, 
                                                  InputDataConfig= {"DatasetGroupArn": datasetGroupArn},
                                                  FeaturizationConfig= {"ForecastFrequency": "H", 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )

In [ ]:
predictor_arn=create_predictor_response['PredictorArn']

以下のセルに実行には約25分かかります（アルゴリズムやAutoM、HPOの有無によって実行時間は異なります）。

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_predictor(PredictorArn=predictor_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

In [ ]:
# 評価指標の取得
forecast.get_accuracy_metrics(PredictorArn=predictor_arn)

### Forecastの作成

次に、作成したpredictorを使用して予測を作成します

In [ ]:
# Forecastの名前
forecastName= project+'_ets_algo_forecast'

In [ ]:
create_forecast_response=forecast.create_forecast(
    ForecastName=forecastName,
    PredictorArn=predictor_arn)

forecast_arn = create_forecast_response['ForecastArn']

以下のセルに実行には約25分かかります（アルゴリズムによって実行時間は異なります）。

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast(ForecastArn=forecast_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

### 予測値の取得

ここではサンプルとして「client_21」の予測値を取得します。

In [ ]:
print(forecast_arn)
print()
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecast_arn,
    Filters={"item_id":"client_21"}
)
print(forecastResponse)

## 予測値と実測値の可視化

予測値と実測値を可視化してみましょう。

In [ ]:
actual_df = pd.read_csv("./electricityusagedata_test.csv", names=['timestamp','value','item'])
actual_df.head()

In [ ]:
actual_df = actual_df[(actual_df['timestamp'] >= '2014-12-30 00:00:00') & (actual_df['timestamp'] <= '2014-12-31 00:00:00')]

In [ ]:
actual_df = actual_df[(actual_df['item'] == 'client_21')]
actual_df.head()

In [ ]:
# 実測値のplot
actual_df.plot()

In [ ]:
# 10パーセンタイル点の予測値を取得します
prediction_df_p10 = pd.DataFrame.from_dict(forecastResponse['Forecast']['Predictions']['p10'])
prediction_df_p10.head()

In [ ]:
# Plot
prediction_df_p10.plot()

In [ ]:
# 同様に50, 90パーセンタイル点の値を取得します
prediction_df_p50 = pd.DataFrame.from_dict(forecastResponse['Forecast']['Predictions']['p50'])
prediction_df_p90 = pd.DataFrame.from_dict(forecastResponse['Forecast']['Predictions']['p90'])

In [ ]:
# We start by creating a dataframe to house our content, here source will be which dataframe it came from
results_df = pd.DataFrame(columns=['timestamp', 'value', 'source'])

In [ ]:
for index, row in actual_df.iterrows():
    clean_timestamp = dateutil.parser.parse(row['timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['value'], 'source': 'actual'} , ignore_index=True)

In [ ]:
# To show the new dataframe
results_df.head()

In [ ]:
# Now add the P10, P50, and P90 Values
for index, row in prediction_df_p10.iterrows():
    clean_timestamp = dateutil.parser.parse(row['Timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['Value'], 'source': 'p10'} , ignore_index=True)
for index, row in prediction_df_p50.iterrows():
    clean_timestamp = dateutil.parser.parse(row['Timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['Value'], 'source': 'p50'} , ignore_index=True)
for index, row in prediction_df_p90.iterrows():
    clean_timestamp = dateutil.parser.parse(row['Timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['Value'], 'source': 'p90'} , ignore_index=True)

In [ ]:
results_df

In [ ]:
pivot_df = results_df.pivot(columns='source', values='value', index="timestamp")
pivot_df

In [ ]:
# 予測値と実測値のplot
pivot_df.plot()

## RollingForecastの実行
 それでは、先ほど作成したpredictorを使用してさらに24時間先の予測を行ってみましょう。
 
 まずはelectricityusagedata_addに対するデータセットインポートジョブを作成します（追加データでRolling Forecastする場合、追加データには学習に使った既存データが全て含まれている必要があります）。 

In [ ]:
# electricityusagedata_add.csvのパス
s3DataPath = "s3://"+bucket_name+"/"+add_key

In [ ]:
# データセットの定義
new_datasetArn = response['DatasetArn']
forecast.describe_dataset(DatasetArn=new_datasetArn)

In [ ]:
datasetImportJobName = 'add_dataset_import'
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=new_datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [ ]:
new_ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(new_ds_import_job_arn)

以下のセルに実行には約5分かかります。

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_dataset_import_job(DatasetImportJobArn=new_ds_import_job_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

In [ ]:
forecast.describe_dataset_import_job(DatasetImportJobArn=new_ds_import_job_arn)

### 新しいForecast（Rolling Forecast）の作成

作成済みのpredictorを使用して、さらに24時間先の予測値を取得します

In [ ]:
# 新しいForecastの名前
forecastName= project+'_ets_algo_forecast_new'

In [ ]:
create_forecast_response=forecast.create_forecast(
    ForecastName=forecastName,
    PredictorArn=predictor_arn)

# 新しいForecastARN
new_forecast_arn = create_forecast_response['ForecastArn']

以下のセルに実行には約25分かかります。

In [ ]:
status_indicator = util.StatusIndicator()

while True:
    status = forecast.describe_forecast(ForecastArn=new_forecast_arn)['Status']
    status_indicator.update(status)
    if status in ('ACTIVE', 'CREATE_FAILED'): break
    time.sleep(10)

status_indicator.end()

### Rolling Forecastの予測値の取得

先ほどと同様にサンプルとして「client_21」の予測値を取得します。

In [ ]:
print(new_forecast_arn)
print()
forecastResponse = forecastquery.query_forecast(
    ForecastArn=new_forecast_arn,
    Filters={"item_id":"client_21"}
)
print(forecastResponse)

### Rolling Forecastの予測値と実測値の可視化

Rolling Forecastの予測値と実測値を可視化してみましょう。

In [ ]:
actual_df = pd.read_csv("./electricityusagedata_test.csv", names=['timestamp','value','item'])
actual_df.head()

In [ ]:
actual_df = actual_df[(actual_df['timestamp'] >= '2014-12-31 00:00:00') & (actual_df['timestamp'] <= '2015-01-01 00:00:00')]

In [ ]:
actual_df = actual_df[(actual_df['item'] == 'client_21')]
actual_df.head()

In [ ]:
# 10パーセンタイル点の予測値を取得します
prediction_df_p10 = pd.DataFrame.from_dict(forecastResponse['Forecast']['Predictions']['p10'])
prediction_df_p10.head()

In [ ]:
# Plot
prediction_df_p10.plot()

In [ ]:
# 同様に50, 90パーセンタイル点の値を取得します
prediction_df_p50 = pd.DataFrame.from_dict(forecastResponse['Forecast']['Predictions']['p50'])
prediction_df_p90 = pd.DataFrame.from_dict(forecastResponse['Forecast']['Predictions']['p90'])

In [ ]:
# We start by creating a dataframe to house our content, here source will be which dataframe it came from
results_df = pd.DataFrame(columns=['timestamp', 'value', 'source'])

In [ ]:
for index, row in actual_df.iterrows():
    clean_timestamp = dateutil.parser.parse(row['timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['value'], 'source': 'actual'} , ignore_index=True)

In [ ]:
# To show the new dataframe
results_df.head()

In [ ]:
# Now add the P10, P50, and P90 Values
for index, row in prediction_df_p10.iterrows():
    clean_timestamp = dateutil.parser.parse(row['Timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['Value'], 'source': 'p10'} , ignore_index=True)
for index, row in prediction_df_p50.iterrows():
    clean_timestamp = dateutil.parser.parse(row['Timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['Value'], 'source': 'p50'} , ignore_index=True)
for index, row in prediction_df_p90.iterrows():
    clean_timestamp = dateutil.parser.parse(row['Timestamp'])
    results_df = results_df.append({'timestamp' : clean_timestamp , 'value' : row['Value'], 'source': 'p90'} , ignore_index=True)

In [ ]:
results_df

In [ ]:
pivot_df = results_df.pivot(columns='source', values='value', index="timestamp")
pivot_df

In [ ]:
pivot_df.plot()

青線が実測値で黄線、緑線、赤線がそれぞれ10、50、90パーセンタイルの予測値になります。深夜帯の変化のない部分、夕方の上昇、夜間の減少といった変化を予測できていることを確認できました。

## リソースの削除

Forecastで作成したリソースとs3にアップロードしたオブジェクトを削除します。    
これらの操作はForecastおよびs3のコンソール上からも実行可能です。

In [ ]:
# Delete the Foreacst:
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=forecast_arn))

In [ ]:
# Delete the Rolling Foreacst:
util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn=new_forecast_arn))

In [ ]:
# Delete the Predictor:
util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn=predictor_arn))

In [ ]:
# Delete ds_import_job_arn
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ds_import_job_arn))

In [ ]:
# Delete new_ds_import_job_arn
util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=new_ds_import_job_arn))

In [ ]:
# Delete the datasetArn:
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=datasetArn))

In [ ]:
# Delete the new_datasetArn:
util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=new_datasetArn))

In [ ]:
# Delete the DatasetGroup:
util.wait_till_delete(lambda: forecast.delete_dataset_group(DatasetGroupArn=datasetGroupArn))

In [ ]:
# Delete train file in S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).delete()

In [ ]:
# Delete add file in S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(add_key).delete()

In [ ]:
# Delete your S3 bucket(以下のコードの実行はs3のFull Accessが必要です。権限がない場合はs3のコンソールから行ってください。)
boto3.Session().resource('s3').Bucket(bucket_name).delete()

### IAM RoleとPolicyの削除

In [ ]:
util.delete_iam_role(role_name)